In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from datetime import datetime
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

checking dataset size

In [1]:
! ls -la ../input/
! wc -l ../input/train.csv
! wc -l ../input/test.csv

train and test dataset is split by click_time
- train: 2017-11-06 - 2017-11-09
- test : 2017-11-10 - 2017-11-11

In [3]:
! head ../input/train.csv
! tail ../input/train.csv

In [4]:
! head ../input/test.csv
! tail ../input/test.csv

In [5]:
def read_file(train_or_test, is_sample=False, skiprows=None, nrows=None):
    sample = '_sample' if is_sample else ''
    df = pd.read_csv(
                        '../input/{}{}.csv'.format(train_or_test, sample),
                        dtype={'attributed_time': str, 'click_time': str}, parse_dates=['click_time', 'attributed_time'],
                        skiprows=skiprows, low_memory=True, nrows=nrows)
    
    print(train_or_test, df.shape)
    display(df.head(2))
    return df

def describe_data(df):
    display(pd.concat((df.describe(), pd.DataFrame({'dtype': df.dtypes, 'nunique': df.nunique(), 'isnull': np.sum(df.isnull())}).T)))

1. read just 10,000,000 rows which is less than 5 % of train datasets

In [6]:
df_tr = read_file('train', is_sample=False, nrows=10000000)
# df_tr = read_file('train', is_sample=False, skiprows=np.random.randint(0, 1000000, 100000))

In [ ]:
describe_data(df_tr)

is_attributed balance

In [ ]:
df_tr.is_attributed.value_counts().plot(kind='bar', log=True)

In [ ]:
df_tr.ip.value_counts().hist(bins=100, normed=False, log=True)

In [ ]:
df_tr.app.value_counts().hist(bins=100, normed=False, log=True)

In [ ]:
df_tr.channel.value_counts().hist(bins=100, normed=False, log=True)

In [ ]:
df_tr.groupby('channel', as_index=False)['is_attributed'].mean().plot(kind='bar', x='channel', y='is_attributed', figsize=(24, 4), log=True)

In [ ]:
df_tr.groupby('device', as_index=False)['is_attributed'].mean().plot(kind='bar', x='device', y='is_attributed', figsize=(24, 4), log=True)

In [ ]:
df_tr.groupby('app', as_index=False)['is_attributed'].mean().plot(kind='bar', x='app', y='is_attributed', figsize=(24, 4), log=True)

In [ ]:
df_tr.groupby('os', as_index=False)['is_attributed'].mean().plot(kind='bar', x='os', y='is_attributed', figsize=(24, 4), log=True)

In [ ]:
_df_tr_g = df_tr.groupby(['ip', 'channel'], as_index=False)['is_attributed'].sum()
# _df_tr_g[_df_tr_g.is_attributed > 0].plot(kind='bar', x=['ip', 'channel'], y='is_attributed', figsize=(24, 4), log=False)
_df_tr_g[_df_tr_g.is_attributed > 0]

[](http://)duration from clicking time to attributed time

In [ ]:
df_tr_tmp = df_tr[~df_tr.attributed_time.isnull()]
df_tr_tmp['time_diff'] = df_tr_tmp['attributed_time'] - df_tr_tmp['click_time']
df_tr_tmp['time_diff'] =df_tr_tmp['time_diff'].map(lambda x: x.seconds / 3600.)
display(df_tr_tmp.head(2))
plt.title('hours duration')
df_tr_tmp.time_diff.hist(bins=100, log=True, figsize=(24, 4))



There are some cycle of attributed with hour in days

In [ ]:
df_tr_tmp = df_tr[~df_tr.attributed_time.isnull()]
df_tr_tmp['attributed_hour'] =df_tr_tmp.attributed_time.dt.hour
df_tr_tmp['attributed_date'] =df_tr_tmp.attributed_time.dt.strftime("%Y-%m-%d")
display(df_tr_tmp.head(2))
df_tr_tmp.groupby('attributed_hour')['is_attributed'].sum().plot(kind='bar')  
df_tr_tmp.groupby(['attributed_date', 'attributed_hour'])['is_attributed'].sum().plot(kind='bar', log=False, figsize=(24, 4))

In [ ]:
df_tr.groupby(['ip', 'is_attributed'])['click_time'].count()

pt = pd.pivot_table(df_tr,
                        index=['ip'], 
                        columns=['is_attributed'],
                        values=['click_time'],
                        aggfunc=lambda x : len(x),
                        fill_value=0).reset_index(drop=True)

In [ ]:
display(df_tr.groupby(['channel'], as_index=False)['is_attributed'].sum().corr())